In [ ]:
from keras.layers               import Input, Dense, GRU, LSTM, RepeatVector
from keras.models               import Model, Sequential
from keras.layers.core          import Flatten
from keras.callbacks            import LambdaCallback
from keras.optimizers           import SGD, RMSprop, Adam
from keras.layers.wrappers      import Bidirectional as Bi
from keras.layers.wrappers      import TimeDistributed as TD
from keras.layers               import merge
from keras.applications.vgg16   import VGG16 
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers.normalization import BatchNormalization as BN
from keras.layers.noise         import GaussianNoise as GN
import numpy as np
import sys, os, io, json, re
import random
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
"""学習データの準備"""

import MeCab
import pandas as pd

# m = MeCab.Tagger('-Ochasen')
m = MeCab.Tagger(' -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
m.parse('')  #バグ回避

data = pd.read_csv('boketeScrapy/legend.csv')

In [ ]:
"""
テキスト抽出

　形態素解析してリスト形式にする
"""

import mojimoji

w_ys = []
for txt in data['txt']:
    txt_list = txt.split(",")
    for txt in txt_list:
        # 形態素解析
        ys = []
        tmp_ys = []
        only_symbol = True
        node = m.parseToNode(txt)
        while node:
            # surfaceで単語、featureで解析結果が取得できる
            surface = mojimoji.han_to_zen(node.surface, digit=False)
            features = node.feature.split(",")
            # feature = [品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音]
            if features[0] == "BOS/EOS" and features[-3] == "*":
                node = node.next
                continue
            if features[0] == "記号":
                # 基本的に記号がキレイに処理できない様なので無視するが、形態素解析結果が全て記号だった時はそのまま使用する
                tmp_ys.append(surface)
            else:
                ys.append(surface)
                only_symbol = False
            node = node.next

        if only_symbol is True:
            tmp_ys.append("EOS")
            ys = tmp_ys
        else:
            ys.append("EOS")
        w_ys.append(ys)

# print(w_ys)

In [ ]:
'''
単語のIndex作成

形態素解析した単語の一覧辞書を作る
'''

import collections

corpus = []
for word in w_ys:
    corpus.extend(word)

"""
登場回数降順のタプルにする
collections.Counter(corpus).items(): 単語がkey, 出現回数がvalueとなる辞書を作り、[(key, val), (key, val), ...]に変換
key=lambda x:x[1]: sortedでvalをソートキーに指定
reverse=True: 降順
"""
word_id = [ (k,v) for k,v in sorted(collections.Counter(corpus).items(), key=lambda x:x[1], reverse=True) if k!="EOS" ]
# {単語:（１から始まる）ID}という辞書を作成
word_id = { k:e for e, (k,v) in enumerate(word_id, start = 1)}
word_id["EOS"] = 0
# print(word_id)

In [ ]:
'''
逆引き辞書作成

{ID: 単語}という辞書を作成
'''

id_word = {v:k for k,v in word_id.items()}
# print(id_word)

In [ ]:
print("All words:\t\t", len(word_id))
# 平均単語数
print("Avg. boke length:\t", np.mean(np.array([len(w) for w in w_ys])))

In [ ]:
'''
単語リストのパディング

MAX_LEN = 最大単語数
（上記の平均単語数を元に決める）
''' 

MAX_LEN = 20

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

# 形態素解析した単語を全てIDに変換する
id_ys = [[word_id[w] for w in ws] for ws in w_ys]

# MAX_LENを超えた要素に対して、超過分のtruncate、不足分を"EOS"でpddingを行う処理
# 返り値は２次元numpy配列
vec_ys = pad_sequences(id_ys, maxlen=MAX_LEN, padding='post', truncating="post", value=word_id["EOS"])

print("vector shape:\t",vec_ys.shape)

In [ ]:
'''
単語リストをone-hot(0-1)化
'''

from keras.utils import to_categorical

# print(vec_ys)
# [277 114 147   6 236 251   9  46  86  13]

# to_categorical(): クラスベクトルa（0からlen(word_id)までの整数）を categorical_crossentropyとともに用いるためのバイナリのクラス行列に変換
Y = np.array([to_categorical(a, len(word_id)) for a in vec_ys])
# print(Y)

In [ ]:
"""画像読み込み"""
X = []
for image_txt in data['images']:
    image_list = image_txt.strip('[]')
    images = re.findall(r'{.+?}', image_list)
    for image in images:
        image = image.replace("'", '"')
        image = json.loads(image)
        path = 'boketeScrapy/img/' + image['path']
        img = load_img(path, target_size=(150, 150))
        x = img_to_array(img)
        X.append(x)

X = np.asarray(X)
print(X.shape)

In [ ]:
"""
モデル生成

Getting started with the Keras functional API: https://keras.io/ja/getting-started/functional-api-guide/
"""

units = 1024

input_tensor = Input(shape=(150, 150, 3))
vgg_model = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)
vgg_x = vgg_model.layers[-1].output
vgg_x = Flatten()(vgg_x)
vgg_x = Dense(units)(vgg_x)

timestep = MAX_LEN
# 入力が文章ならRepeatVectorの数は単語数になるが、今回は画像なので適正値が不明
inputs = RepeatVector(MAX_LEN)(vgg_x)
encoded = LSTM(units)(inputs)
encoder = Model(input_tensor, encoded)

DIM = len(word_id)
x = RepeatVector(MAX_LEN)(encoded)
x = Bi(LSTM(units, return_sequences=True))(x)
# 入力次元は単語辞書と同じサイズになる
decoded = TD(Dense(DIM, activation='softmax'))(x)

model = Model(input_tensor, decoded)

# for i, layer in enumerate(model.layers): # default 15
#   print( i, layer )

for layer in model.layers[:18]: # 18 is max of VGG16
  layer.trainable = False

model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])
#  下記も検証してみる
# optimizer=' rmsprop'
# loss=binary_crossentropy

model.summary()

In [ ]:
"""学習実行"""
epoch_cnt = 1000
batch_size = 32  # データ数が少ないので、kerasのdefaultをそのまま使用

path = 'model/weights.{epoch:02d}-{loss:.2f}-{acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(path, monitor='loss', verbose=1, save_best_only=True, mode='auto')
early_stopping = EarlyStopping(monitor='loss', patience=6, verbose=1, mode='auto')
# history = model.fit(X, Y,  shuffle=True, batch_size=batch_size, epochs=epoch_cnt, verbose=1, validation_split=0.1, callbacks=[model_checkpoint, early_stopping] )
history = model.fit(X, Y,  shuffle=True, batch_size=batch_size, epochs=epoch_cnt, verbose=1, callbacks=[model_checkpoint, early_stopping] )

In [ ]:
# 学習結果をグラフ表示

import matplotlib.pyplot as plt
# plot results
loss = history.history['loss']
# val_loss = history.history['val_loss']

acc = history.history['acc']
# val_acc = history.history['val_acc']

plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.title('Loss')
epochs = len(loss)
plt.plot(range(epochs), loss, marker='.', label='loss')
# plt.plot(range(epochs), val_loss, marker='.', label='val_loss')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')

plt.subplot(2,1,2)
plt.title('Accuracy')
plt.plot(range(epochs), acc, marker='.', label='acc')
# plt.plot(range(epochs), val_acc, marker='.', label='val_acc')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show()

In [ ]:
"""学習済みモデルをロード"""

model.load_weights('')

In [ ]:
# ボケ実行

# 画像をnp.array()にする
# 未学習画像
# f = "f5b031451d0180c14f14844a48b97f08_600"
# path_test = "img/" + f + ".jpg"

# スクレイピングしてきた画像
f = "0c07c5fa50ed68ac2a84db5b134cf6a06d4b2276"
path_test = "boketeScrapy/img/full/" + f  + ".jpg"

X_test = []
img_test = load_img(path_test, target_size=(150, 150))
x_test = img_to_array(img_test)
X_test.append(x_test)
X_test = np.asarray(X_test)

# 予測実行
pred = model.predict(X_test)
print(pred.shape)
print(pred[0])

# 予測結果を元にid_wordから文字列を抽出
for p in pred[0]:
    if np.argmax(p) == 0:
        continue
    print(id_word[int(np.argmax(p))], end=" ")